In [ ]:
%tensorflow_version 2.x

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import sys
import os
sys.path.append('/content/gdrive/My Drive/W266-NLP/Project')
os.chdir('/content/gdrive/My Drive/W266-NLP/Project')
os.listdir()

In [1]:
import os
import time
import numpy as np
import pickle
import pandas as pd
import json
from pandas.io.json import json_normalize
from collections import Counter
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Attention
import nltk
from functools import reduce
!pip install wget
# Load PyDrive and Google Auth related packages
#!pip install -U -q PyDrive
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Attention
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input
from functools import reduce
flatten = lambda data: reduce(lambda x, y: x + y, data)
import glove_helper

# Load the json data
def load_json_file(name):
  """
  Load the json file and return a json object
  """
  with open(name,encoding='utf-8') as myfile:
    data = json.load(myfile)
    return data

# Convert json data object to a pandas data frame
def convert_to_pd(data):
  """
  Load the data to a pandas dataframe.
  Dataframe Columns:
    title
    para_index
    context
    q_index
    q_id
    q_isimpossible
    q_question
    q_anscount - number of answers
    q_answers - a list of object e.g [{ text: '', answer_start: 123}, ...]
  """
  result = []
  for pdata in data['data']:
    for para in pdata['paragraphs']:
      for q in para['qas']:
        result.append({
            'title' : pdata['title'],
            'context' : para['context'],
            'q_id' : q['id'],
            'q_isimpossible' : q['is_impossible'],
            'q_question' : q['question'],
            'q_anscount' : len(q['answers']),
            'q_answers' : [a for a in q['answers']],
            'q_answers_text': [a.get("text") for a in q['answers']],
            'context_lowercase': para['context'].lower(),
            'q_question_lowercase' : q['question'].lower(),
            'q_answers_text_lowercase': [a.get("text").lower() for a in q['answers']],
            
        })

  return pd.DataFrame.from_dict(result, orient='columns')

# Load the file from shareable google drive link and return a pandas dataframe
def loadDataFile(filename): 
  """
  Download a file from google drive with the shared link
  """ 
  data = load_json_file(filename)
  return convert_to_pd(data)

In [2]:
# DONOT RUN THIS ON COLAB#
#to make use of CPU and not GPU DONOT RUN THIS ON COLAB
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""


In [2]:
train_filename = 'train-v2.0.json'
dev_filename = 'dev-v2.0.json'

train_pd = loadDataFile(train_filename)
dev_pd = loadDataFile(dev_filename)

In [3]:
def get_c_q_a(dataset):
    q_id_list = []
    context_list =[]
    questions_list = []
    answers_list =[]
    q_impossible_list =[]
    for index,row in dataset.iterrows():
        q_id_list.append(row.q_id)
        context_list.append(row.context)
        questions_list.append(row.q_question)
        q_impossible_list.append(int(row.q_isimpossible))
        if len(row.q_answers_text)>0 :
            answers_list.append(row.q_answers_text[0])
        else:
            answers_list.append("")
    return [q_id_list,context_list,questions_list,q_impossible_list,answers_list]

train_lists = get_c_q_a(train_pd)
dev_lists = get_c_q_a(dev_pd)
context_maxlen = max(map(len, (x.split() for x in train_lists[1])))
question_maxlen = max(map(len, (x.split() for x in train_lists[2])))
answer_maxlen = max(map(len, (x.split() for x in train_lists[4])))
print("Max context length:",context_maxlen)
print("Max question length:",question_maxlen)
print("Max answer length:",answer_maxlen)

Max context length: 653
Max question length: 40
Max answer length: 43


In [4]:
context_maxlen = 214
question_maxlen = 18
answer_maxlen = 10

In [5]:
def tokenize_c_q_a(dataset,num_words=None):
    tokenizer = Tokenizer(num_words,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"''",oov_token='<unk>')
    data = dataset[1]+dataset[2]+dataset[4]
    tokenizer.fit_on_texts(data)
    vocab = {}
    for word,i in tokenizer.word_index.items():
        if num_words is not None:
          if i <= num_words:
            vocab[word] = i
        else:
          vocab[word] = i
    #vocab = tokenizer.word_index
    vocab['<s>'] = len(vocab)+1
    vocab['</s>'] = len(vocab)+1
    id_vocab = {value: key for key, value in vocab.items()}
    return (tokenizer,vocab,id_vocab)

tokenizer_obj,vocab,id_vocab = tokenize_c_q_a(train_lists)

vocab_size = len(vocab)
print("Vocab Size:",vocab_size)

def vectorize_data(tokenizer_obj,train_lists):
    context_seq = tokenizer_obj.texts_to_sequences(train_lists[1])
    question_seq = tokenizer_obj.texts_to_sequences(train_lists[2])
    answer_seq = tokenizer_obj.texts_to_sequences(train_lists[4])
    answer_input_seq = [[vocab['<s>']]+i+[vocab['</s>']] for i in answer_seq]
    answer_target_seq = [i+[vocab['</s>']] for i in answer_seq]
    context_seq_padded = pad_sequences(context_seq,context_maxlen,padding='post', truncating='post')
    question_seq_padded = pad_sequences(question_seq,question_maxlen,padding='post', truncating='post')
    answer_seq_padded = pad_sequences(answer_seq,answer_maxlen,padding='post', truncating='post')
    answer_input_seq_padded = pad_sequences(answer_input_seq,answer_maxlen+2,padding='post', truncating='post')
    answer_target_seq_padded = pad_sequences(answer_target_seq,answer_maxlen+2,padding='post', truncating='post')
    answer_impossible = np.array(train_lists[3])
    indices = np.arange(context_seq_padded.shape[0])
    np.random.seed(19)
    np.random.shuffle(indices)
    context_seq_padded = context_seq_padded[indices]
    question_seq_padded = question_seq_padded[indices]
    answer_seq_padded = answer_seq_padded[indices]
    answer_input_seq_padded = answer_input_seq_padded[indices]
    answer_target_seq_padded = answer_target_seq_padded[indices]
    answer_impossible_shuffled = answer_impossible[indices]
    train_samples = int(((context_seq_padded.shape[0]*.8)//128)*128)
    end_samples = int((context_seq_padded.shape[0]//128)*128)
    train_context_padded_seq = context_seq_padded[:train_samples]
    train_question_seq_padded = question_seq_padded[:train_samples]
    train_answer_seq_padded = answer_seq_padded[:train_samples]
    train_answer_input_seq_padded = answer_input_seq_padded[:train_samples]
    train_answer_target_seq_padded = answer_target_seq_padded[:train_samples]
    train_answer_impossible = answer_impossible_shuffled[:train_samples]
    val_context_padded_seq = context_seq_padded[train_samples:end_samples]
    val_question_seq_padded = question_seq_padded[train_samples:end_samples]
    val_answer_seq_padded = answer_seq_padded[train_samples:end_samples]
    val_answer_input_seq_padded = answer_input_seq_padded[train_samples:end_samples]
    val_answer_target_seq_padded = answer_target_seq_padded[train_samples:end_samples]
    val_answer_impossible = answer_impossible_shuffled[train_samples:end_samples]
    return (train_context_padded_seq,train_question_seq_padded,train_answer_seq_padded,
            train_answer_input_seq_padded,train_answer_target_seq_padded,train_answer_impossible,
            val_context_padded_seq,val_question_seq_padded,val_answer_seq_padded,
            val_answer_input_seq_padded,val_answer_target_seq_padded,val_answer_impossible)

train_context_padded_seq,train_question_seq_padded,train_answer_seq_padded,\
train_answer_input_seq_padded,train_answer_target_seq_padded,\
train_answer_impossible,\
val_context_padded_seq,val_question_seq_padded,val_answer_seq_padded,\
val_answer_input_seq_padded,val_answer_target_seq_padded,\
val_answer_impossible\
= vectorize_data(tokenizer_obj,train_lists)

print("validation num samples where answer impossible: ",len(val_answer_seq_padded[val_answer_impossible==1]))
print("validation num samples where answer not impossible: ",len(val_answer_seq_padded[val_answer_impossible==0]))
print("train num samples where answer impossible: ",len(train_answer_seq_padded[train_answer_impossible==1]))
print("train num samples where answer not impossible: ",len(train_answer_seq_padded[train_answer_impossible==0]))

Vocab Size: 88701
validation num samples where answer impossible:  8730
validation num samples where answer not impossible:  17382
train num samples where answer impossible:  34761
train num samples where answer not impossible:  69431


In [6]:
train_question_seq_padded.shape[0]

104192

In [7]:
val_answer_seq_padded.shape[0]

26112

In [8]:
def create_embedding_matrix(word_index,vocab_size=50000,ndim=100):
    hands = glove_helper.Hands(ndim)
    embedding_matrix = np.zeros((vocab_size+1,ndim))
    for word,i in word_index.items():
        if i<=vocab_size:
            embedding_vector = hands.get_vector(word,strict=False)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix
ndim = 100
embedding_matrix = create_embedding_matrix(vocab,vocab_size,ndim)

Loading vectors from data/glove/glove.6B.zip
Parsing file: data/glove/glove.6B.zip:glove.6B.100d.txt
Found 400,000 words.
Parsing vectors... Done! (W.shape = (400003, 100))


In [9]:
#New Model with attention in every step of the answer decoder
class BahdanauAttention_model2(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention_model2, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
                                self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

# Answer Module which is custom as we need to feed output of each time sequence with attention to next 
# time sequence
class answer_module(tf.keras.Model):
    def __init__(self,embedding_matrix,
                      vocab_size,
                      ndim,
                      num_unit_gru,
                      num_layers_gru,
                      dropout_rate,
                      l1_regularizer_weight = .01,
                      l2_regularizer_weight = .01
                      ):
        super(answer_module, self).__init__()
        self.num_unit_gru = num_unit_gru
        self.dropout_rate = dropout_rate
        self.attention_layer = BahdanauAttention_model2(num_unit_gru)
        #self.answer_input_layer = Input(shape=(None,),dtype='int32',name='Answer_Input')
        self.answer_embedding_layer = layers.Embedding(vocab_size+1,
                                                       ndim,
                                                       mask_zero=True,
                                                       weights =[embedding_matrix],
                                                       trainable = False,
                                                       name='Answer_Embedding')
        self.answer_output_layers = []
        self.batch_normalization_layers = []
        for i in range(num_layers_gru):
            self.answer_output_layers.append(layers.GRU(self.num_unit_gru,
                                                        dropout=self.dropout_rate,
                                                        recurrent_dropout= self.dropout_rate,
                                                        return_sequences=True,
                                                        return_state=True,
                                                        recurrent_initializer='glorot_uniform',
                                                        name='Answer_GRU_Layer'+str(i),
                                                        kernel_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight),
                                                        bias_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight)
                                                       )
                                              )
            self.batch_normalization_layers.append(layers.BatchNormalization())
        
        
        self.answer_decoder_dense = layers.Dense(vocab_size+1,
                                                 name='Answer_output',
                                                 kernel_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight),
                                                 bias_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight))
        
    
    def call(self,
             target_input,
             answer_hidden,
             question,
             context,
             episodic_memory):
        attention_output,attention_weights = self.attention_layer(answer_hidden,context)
        answer_embeddings = self.answer_embedding_layer(target_input)
        answer_concat = tf.concat([tf.expand_dims(attention_output, 1),
                                   tf.expand_dims(question, 1),
                                   answer_embeddings], axis=-1)
        for i in range(len(self.answer_output_layers)):
            if ((episodic_memory != None) and (i==0)):
                answer_outputs,hidden_state = self.answer_output_layers[i](answer_concat,
                                                                           initial_state=episodic_memory)
            elif i==0:
                answer_outputs,hidden_state = self.answer_output_layers[i](answer_concat)
            else:
                answer_outputs,hidden_state = self.answer_output_layers[i](answer_outputs)
            
            answer_outputs   = self.batch_normalization_layers[i](answer_outputs)
        answer_outputs = tf.reshape(answer_outputs, (-1, answer_outputs.shape[2]))

        answer_decoder_outputs = self.answer_decoder_dense(answer_outputs)
        return answer_decoder_outputs,hidden_state,attention_weights

#Encoder Module which combines the context,question into episodic memory and emits context outputs and 
#question outputs
class encoder_module(tf.keras.Model):    
    def __init__(self,embedding_matrix,
                      vocab_size,
                      max_context_length,
                      max_question_length,
                      max_answer_length,
                      num_unit_gru = 64,
                      num_layers_gru = 2,
                      ndim =100,
                      num_episodes = 2,
                      dropout_rate = 0.5,
                      num_episodic_network_unit = 64,
                      l1_regularizer_weight = .01,
                      l2_regularizer_weight = .01
                      ):
        super(encoder_module, self).__init__()
        #Context Module
        self.num_unit_gru = num_unit_gru
        self.dropout_rate = dropout_rate
        self.num_layers_gru = num_layers_gru
        self.embedding_matrix = embedding_matrix
        self.vocab_size = vocab_size
        self.ndim = ndim
        self.max_context_length = max_context_length
        self.max_question_length = max_question_length
        self.max_answer_length = max_answer_length
        self.num_episodes = num_episodes
        self.num_episodic_network_unit = num_episodic_network_unit
        self.context_embeddings_layer = layers.Embedding(self.vocab_size+1,
                                                         self.ndim,
                                                         mask_zero=True,
                                                         weights =[self.embedding_matrix],
                                                         trainable = False,
                                                         name='Context_Embedding')
        self.context_output_layers = []
        self.context_batch_normalization_layers = []
        for i in range(self.num_layers_gru):
            self.context_output_layers.append(layers.Bidirectional(layers.GRU(self.num_unit_gru,
                                                                               dropout=self.dropout_rate,
                                                                               recurrent_dropout= self.dropout_rate,
                                                                               recurrent_initializer='glorot_uniform',
                                                                               return_sequences=True,
                                                                               kernel_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight),
                                                                               bias_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight)),
                                                                   merge_mode='sum',
                                                                   name='Context_Bid_Layer'+str(i))
                                         )
            self.context_batch_normalization_layers.append(layers.BatchNormalization())
        
        #Question Module
        self.question_embeddings_layer = layers.Embedding(self.vocab_size+1,
                                                          self.ndim,
                                                          mask_zero=True,
                                                          weights =[self.embedding_matrix],
                                                          trainable = False,
                                                          name='Question_Embedding')
          
        self.question_output_layers = []
        self.question_batch_normalization_layers = []
        for i in range(num_layers_gru):

            if i==0 and num_layers_gru >1:
                self.question_output_layers.append(layers.Bidirectional(
                                                             layers.GRU(self.num_unit_gru,
                                                                        dropout=self.dropout_rate,
                                                                        recurrent_dropout= self.dropout_rate,
                                                                        recurrent_initializer='glorot_uniform',
                                                                        return_sequences=True,
                                                                        kernel_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight),
                                                                        bias_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight)),
                                                             merge_mode='sum',
                                                             name='Question_Bid_Layer'+str(i))
                                              )
            elif i==0 and num_layers_gru ==1:
                self.question_output_layers.append(layers.Bidirectional(
                                                             layers.GRU(self.num_unit_gru,
                                                                        dropout=self.dropout_rate,
                                                                        recurrent_dropout= self.dropout_rate,
                                                                        recurrent_initializer='glorot_uniform',
                                                                        return_sequences=False,
                                                                        kernel_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight),
                                                                         bias_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight)),
                                                             merge_mode='sum',
                                                             name='Question_Bid_Layer'+str(i))
                                             )
            elif i==(num_layers_gru-1):
                self.question_output_layers.append(layers.Bidirectional(
                                                             layers.GRU(self.num_unit_gru,
                                                                        dropout=self.dropout_rate,
                                                                        recurrent_dropout= self.dropout_rate,
                                                                        recurrent_initializer='glorot_uniform',
                                                                        return_sequences=False,
                                                                        kernel_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight),
                                                                        bias_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight)),
                                                              merge_mode='sum',
                                                              name='Question_Bid_Layer'+str(i))
                                             )
            else:
                self.question_output_layers.append(layers.Bidirectional(
                                                             layers.GRU(self.num_unit_gru,
                                                                        dropout=self.dropout_rate,
                                                                        recurrent_dropout= self.dropout_rate,
                                                                        recurrent_initializer='glorot_uniform',
                                                                        return_sequences=True,
                                                                        kernel_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight),
                                                                        bias_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight)),
                                                                        merge_mode='sum',
                                                                        name='Question_Bid_Layer'+str(i))
                                              )
            self.question_batch_normalization_layers.append(layers.BatchNormalization())
        #Episodic Memory 
        self.episodic_weight_layer = layers.Dense(self.num_unit_gru,use_bias=False)
        self.episodic_tanh_layer = layers.Dense(self.num_episodic_network_unit,activation='tanh')
        self.episodic_score_layer = layers.Dense(1)
        
    def call(self,context_input,question_input):
        #context Module
        context_embeddings = self.context_embeddings_layer(context_input)
        for i in range(len(self.context_output_layers)):
            if i==0:
                context_outputs = self.context_output_layers[i](context_embeddings)
            else:
                context_outputs = self.context_output_layers[i](context_outputs)
            context_outputs = self.context_batch_normalization_layers[i](context_outputs)

        #Question Module
        question_embeddings = self.question_embeddings_layer(question_input)
        for i in range(len(self.question_output_layers)):
            if i==0:
                question_outputs = self.question_output_layers[i](question_embeddings)
            else:
                question_outputs = self.question_output_layers[i](question_outputs)
            question_outputs = self.question_batch_normalization_layers[i](question_outputs) 
        #Episodic Memory 
        m = tf.identity(question_outputs)
        for i in range(self.num_episodes):
            m_increased = tf.tile(tf.keras.backend.expand_dims(m,1),
                                  tf.constant([1,self.max_context_length,1],tf.int32))
            q_increased = tf.tile(tf.keras.backend.expand_dims(question_outputs,1),
                                  tf.constant([1,self.max_context_length,1],tf.int32))
            c_mul_q = layers.multiply([context_outputs,q_increased])
            c_mul_m = layers.multiply([context_outputs,m_increased])
            c_minus_q =tf.keras.backend.abs(layers.subtract([context_outputs,q_increased]))
            c_minus_m = tf.keras.backend.abs(layers.subtract([context_outputs,m_increased]))
            c_dot_q = tf.matmul(tf.keras.backend.expand_dims(self.episodic_weight_layer(question_outputs),1), 
                                context_outputs,
                                transpose_b=True)
            c_dot_q = layers.Permute((2,1))(c_dot_q)
            c_dot_m = tf.matmul(tf.keras.backend.expand_dims(self.episodic_weight_layer(m),1), 
                                context_outputs,transpose_b=True)
            c_dot_m = layers.Permute((2,1))(c_dot_m)
            z = tf.concat([context_outputs,
                                    m_increased,
                                    q_increased,
                                    c_mul_q,
                                    c_mul_m,
                                    c_minus_q,
                                    c_minus_m,
                                    c_dot_q,
                                    c_dot_m],axis=-1)
            score = self.episodic_score_layer(self.episodic_tanh_layer(z))
            attention_weights = tf.nn.softmax(score, axis=1)
            m_value = attention_weights * context_outputs
            m = tf.reduce_sum(m_value, axis=1)
        #concatenate episodic memory with question
        concatenated_tensor = tf.concat(values=[m,question_outputs],axis=1)
        return (m,concatenated_tensor,question_outputs,context_outputs)
    
                
#Function to create the Models
def create_models(embedding_matrix,
                  max_context_length,
                  max_question_length,
                  max_answer_length,
                  num_unit_gru = 64,
                  num_layers_gru = 2,
                  ndim =100,
                  num_episodes = 2,
                  num_dense_layer_feasibility_units = 16,
                  dropout_rate = 0.5,
                  num_dense_layers_feasibility = 1,
                  num_episodic_network_unit = 64,
                  l1_regularizer_weight = .01,
                  l2_regularizer_weight = .01):
    """
    
    def create_episodic_memory(num_episodes,
                               query,
                               context_outputs,
                               max_context_length,
                               max_question_length,
                               num_episodic_network_unit):
        m = layers.Lambda(lambda x: x)(query)
        weight_layer = layers.Dense(query.shape[1],use_bias=False)
        for i in range(num_episodes):
            m_increased = tf.tile(tf.keras.backend.expand_dims(m,1),
                                  tf.constant([1,max_context_length,1],tf.int32))
            q_increased = tf.tile(tf.keras.backend.expand_dims(query,1),
                                  tf.constant([1,max_context_length,1],tf.int32))
            c_mul_q = layers.multiply([context_outputs,q_increased])
            c_mul_m = layers.multiply([context_outputs,m_increased])
            c_minus_q =tf.keras.backend.abs(layers.subtract([context_outputs,q_increased]))
            c_minus_m = tf.keras.backend.abs(layers.subtract([context_outputs,m_increased]))
            c_dot_q = tf.matmul(tf.keras.backend.expand_dims(weight_layer(query),1), 
                                context_outputs,transpose_b=True)
            c_dot_q = layers.Permute((2,1))(c_dot_q)
            c_dot_m = tf.matmul(tf.keras.backend.expand_dims(weight_layer(m),1), context_outputs,transpose_b=True)
            c_dot_m = layers.Permute((2,1))(c_dot_m)
            z = layers.concatenate([context_outputs,
                                    m_increased,
                                    q_increased,
                                    c_mul_q,
                                    c_mul_m,
                                    c_minus_q,
                                    c_minus_m,
                                    c_dot_q,
                                    c_dot_m],axis=-1)
            score = layers.Dense(1)(layers.Dense(num_episodic_network_unit,activation='tanh')(z))
            attention_weights = tf.nn.softmax(score, axis=1)
            m_value = attention_weights * context_outputs
            m = tf.reduce_sum(m_value, axis=1)
        return m
    
    
    #Input Module
    context_input = Input(shape=(None,),dtype='int32',name='Context_Input')
    context_embeddings = layers.Embedding(vocab_size+1,
                                          ndim,
                                          mask_zero=True,
                                          name='Context_Embedding')(context_input)

    for i in range(num_layers_gru):
        context_outputs_layers = layers.Bidirectional(layers.GRU(num_unit_gru,
                                                                 dropout=dropout_rate,
                                                                 recurrent_dropout= dropout_rate,
                                                                 recurrent_initializer='glorot_uniform',
                                                                 return_sequences=True),
                                                      merge_mode='sum',
                                                      name='Context_Bid_Layer'+str(i))
        if i==0:
            context_outputs = context_outputs_layers(context_embeddings)
        else:
            context_outputs = context_outputs_layers(context_outputs)
        context_outputs = layers.BatchNormalization()(context_outputs)
    print("Context output shape",context_outputs.shape)
    #Question Module
    question_input = Input(shape=(None,),dtype='int32',name='Question_Input')
    question_embeddings = layers.Embedding(vocab_size+1,
                                           ndim,
                                           mask_zero=True,
                                           name='Question_Embedding')(question_input)

    for i in range(num_layers_gru):
        if i==0 and num_layers_gru >1:
            question_outputs = layers.Bidirectional(layers.GRU(num_unit_gru,
                                                               dropout=dropout_rate,
                                                               recurrent_dropout= dropout_rate,
                                                               recurrent_initializer='glorot_uniform',
                                                               return_sequences=True),
                                                    merge_mode='sum',
                                                    name='Question_Bid_Layer'+str(i))(question_embeddings)
        elif i==0 and num_layers_gru ==1:
            question_outputs = layers.Bidirectional(layers.GRU(num_unit_gru,
                                                               dropout=dropout_rate,
                                                               recurrent_dropout= dropout_rate,
                                                               recurrent_initializer='glorot_uniform',
                                                               return_sequences=False),
                                                    merge_mode='sum',
                                                    name='Question_Bid_Layer'+str(i))(question_embeddings)
        elif i==(num_layers_gru-1):
            question_outputs = layers.Bidirectional(layers.GRU(num_unit_gru,
                                                               dropout=dropout_rate,
                                                               recurrent_dropout= dropout_rate,
                                                               recurrent_initializer='glorot_uniform',
                                                               return_sequences=False),
                                                    merge_mode='sum',
                                                    name='Question_Bid_Layer'+str(i))(question_outputs)
        else:
            question_outputs = layers.Bidirectional(layers.GRU(num_unit_gru,
                                                               dropout=dropout_rate,
                                                               recurrent_dropout= dropout_rate,
                                                               recurrent_initializer='glorot_uniform',
                                                               return_sequences=True),
                                                    merge_mode='sum',
                                                    name='Question_Bid_Layer'+str(i))(question_outputs)
        question_outputs = layers.BatchNormalization()(question_outputs)
    #Episodic Memory Module
    m=create_episodic_memory(num_episodes,
                             question_outputs,
                             context_outputs,
                             max_context_length,
                             max_question_length,
                             num_episodic_network_unit)

    concatenated_tensor = layers.concatenate(inputs=[m,question_outputs],
                                             name='Concatenation_Memory_Question',axis=1)
    """
    #encoder Model
    encoder_model = encoder_module(embedding_matrix,
                                   vocab_size,
                                   max_context_length,
                                   max_question_length,
                                   max_answer_length,
                                   num_unit_gru,
                                   num_layers_gru,
                                   ndim,
                                   num_episodes,
                                   dropout_rate,
                                   num_episodic_network_unit,
                                   l1_regularizer_weight = l1_regularizer_weight,
                                   l2_regularizer_weight = l2_regularizer_weight
                                 )
    
    
    #Model([context_input,question_input], [m,concatenated_tensor,question_outputs,context_outputs])
    #answer_module
    answer_model = answer_module(embedding_matrix,
                                 vocab_size,
                                 ndim,
                                 num_unit_gru,
                                 num_layers_gru,
                                 dropout_rate,
                                 l1_regularizer_weight = l1_regularizer_weight,
                                 l2_regularizer_weight = l2_regularizer_weight)
    #encoder_model.get_layer("Question_Embedding").set_weights([embedding_matrix])
    #encoder_model.get_layer("Question_Embedding").trainable = False
    #encoder_model.get_layer("Context_Embedding").set_weights([embedding_matrix])
    #encoder_model.get_layer("Context_Embedding").trainable = False
    
    #feasibility module
    feasibility_input = Input(shape=(2*num_unit_gru,), name="FeasibilityInput")
    feasibility_context_input = Input(shape=(None,num_unit_gru,),name='feasibilityContext_Input')
    feasibility_question_input = Input(shape=(num_unit_gru,),name='feasibilityQuestion_Input')
    for i in range(num_dense_layers_feasibility):
        #create attention between Context and Question
        q_with_time_axis = tf.keras.backend.expand_dims(feasibility_question_input,1)
        attentionContextQuestion = layers.AdditiveAttention()([q_with_time_axis,
                                                               feasibility_context_input])
        attentionContextQuestionReduced = tf.keras.backend.sum(attentionContextQuestion, axis=1)
        feasibility_dense_input = tf.concat([feasibility_input,
                                             attentionContextQuestionReduced],
                                            axis=-1)
        
        if i==0:
            dense_layer = layers.Dense(num_dense_layer_feasibility_units,
                                       activation='relu',
                                       name='feasibility_layer_'+str(i),
                                       kernel_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight),
                                        bias_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight)
                                      )(feasibility_dense_input)
        else:
            dense_layer = layers.Dense(num_dense_layer_feasibility_units,
                                       activation='relu',
                                       name='feasibility_layer_'+str(i),
                                       kernel_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight),
                                        bias_regularizer=tf.keras.regularizers.l1_l2(
                                                                                      l1_regularizer_weight,
                                                                                      l2_regularizer_weight)
                                      )(dense_layer)
        dense_layer = layers.BatchNormalization()(dense_layer)
        dropout_layer = layers.Dropout(dropout_rate,name='feasibility_drop_'+str(i))(dense_layer)

    feasibility_output = layers.Dense(1,activation='sigmoid',
                                      name='feasibility_output',
                                      kernel_regularizer=tf.keras.regularizers.l1_l2(
                                                                                     l1_regularizer_weight,
                                                                                    l2_regularizer_weight),
                                      bias_regularizer=tf.keras.regularizers.l1_l2(
                                                                                   l1_regularizer_weight,
                                                                                   l2_regularizer_weight))(dropout_layer)
    feasibility_model = Model([feasibility_input,feasibility_context_input,feasibility_question_input],
                              feasibility_output)
    
    return (answer_model,encoder_model,feasibility_model)

In [10]:
#Function to get sentences from the predicted answers
def decode_sentence(context_input_seq,
                     question_input_seq,
                     encoder_model,
                     answer_model):
    decoded_sentence = ''
    episodic_memory,\
    concatenated_tensor,\
    question_output,\
    context_output = encoder_model(context_input_seq,question_input_seq) 
    answer_hidden = question_output
    dec_input = tf.expand_dims([vocab["<s>"]], 0)

    for t in range(answer_maxlen):
        predictions, answer_hidden, _ = answer_model(dec_input,
                                                     answer_hidden,
                                                     question_output,
                                                     context_output,
                                                     episodic_memory)
        sampled_token_index = tf.argmax(predictions[0]).numpy()
        
        if sampled_token_index == 0:
            sampled_char = " "
        else:
            sampled_char = id_vocab[sampled_token_index]
        decoded_sentence += sampled_char + " "
        if id_vocab[sampled_token_index] == '</s>':
            return decoded_sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([sampled_token_index], 0)
        episodic_memory = None

    return decoded_sentence

In [11]:
Experiment_Dic = {'Experiment0': {'num_unit_gru': 64,
                                  'num_layers_gru': 2,
                                  'num_episodes': 3,
                                  'num_dense_layer_feasibility_units': 32,
                                  'dropout_rate': 0.6,
                                  'num_dense_layers_feasibility': 1,
                                  'num_episodic_network_unit': 192,
                                  'learning_rate': 0.001,
                                  'l1_regularizer_weight':0.01,
                                  'l2_regularizer_weight':0.01},
                  'Experiment1': {'num_unit_gru': 80,
                                  'num_layers_gru': 2,
                                  'num_episodes': 2,
                                  'num_dense_layer_feasibility_units': 64,
                                  'dropout_rate': 0.7,
                                  'num_dense_layers_feasibility': 3,
                                  'num_episodic_network_unit': 192,
                                  'learning_rate': 0.005,
                                  'l1_regularizer_weight':0.0001,
                                  'l2_regularizer_weight':0.01},
                  'Experiment2': {'num_unit_gru': 100,
                                  'num_layers_gru': 2,
                                  'num_episodes': 3,
                                  'num_dense_layer_feasibility_units': 64,
                                  'dropout_rate': 0.5,
                                  'num_dense_layers_feasibility': 1,
                                  'num_episodic_network_unit': 64,
                                  'learning_rate': 0.005,
                                  'l1_regularizer_weight':0.01,
                                  'l2_regularizer_weight':0.01},
                  'Experiment3': {'num_unit_gru': 80,
                                  'num_layers_gru': 2,
                                  'num_episodes': 2,
                                  'num_dense_layer_feasibility_units': 32,
                                  'dropout_rate': 0.7,
                                  'num_dense_layers_feasibility': 3,
                                  'num_episodic_network_unit': 192,
                                  'learning_rate': 0.005,
                                  'l1_regularizer_weight':0.01,
                                  'l2_regularizer_weight':0.0001},
                  'Experiment4': {'num_unit_gru': 80,
                                  'num_layers_gru': 2,
                                  'num_episodes': 2,
                                  'num_dense_layer_feasibility_units': 48,
                                  'dropout_rate': 0.6,
                                  'num_dense_layers_feasibility': 1,
                                  'num_episodic_network_unit': 128,
                                  'learning_rate': 0.005,
                                  'l1_regularizer_weight':0.001,
                                  'l2_regularizer_weight':0.001},
                  'Experiment5': {'num_unit_gru': 80,
                                  'num_layers_gru': 2,
                                  'num_episodes': 3,
                                  'num_dense_layer_feasibility_units': 64,
                                  'dropout_rate': 0.5,
                                  'num_dense_layers_feasibility': 1,
                                  'num_episodic_network_unit': 192,
                                  'learning_rate': 0.005,
                                  'l1_regularizer_weight':0.001,
                                  'l2_regularizer_weight':0.0001}
                }
Experiment_Dic

{'Experiment0': {'num_unit_gru': 64,
  'num_layers_gru': 2,
  'num_episodes': 3,
  'num_dense_layer_feasibility_units': 32,
  'dropout_rate': 0.6,
  'num_dense_layers_feasibility': 1,
  'num_episodic_network_unit': 192,
  'learning_rate': 0.001,
  'l1_regularizer_weight': 0.01,
  'l2_regularizer_weight': 0.01},
 'Experiment1': {'num_unit_gru': 80,
  'num_layers_gru': 2,
  'num_episodes': 2,
  'num_dense_layer_feasibility_units': 64,
  'dropout_rate': 0.7,
  'num_dense_layers_feasibility': 3,
  'num_episodic_network_unit': 192,
  'learning_rate': 0.005,
  'l1_regularizer_weight': 0.0001,
  'l2_regularizer_weight': 0.01},
 'Experiment2': {'num_unit_gru': 100,
  'num_layers_gru': 2,
  'num_episodes': 3,
  'num_dense_layer_feasibility_units': 64,
  'dropout_rate': 0.5,
  'num_dense_layers_feasibility': 1,
  'num_episodic_network_unit': 64,
  'learning_rate': 0.005,
  'l1_regularizer_weight': 0.01,
  'l2_regularizer_weight': 0.01},
 'Experiment3': {'num_unit_gru': 80,
  'num_layers_gru': 2,

In [12]:
def run_experiments(Experiment_Dic,
                    Experiment_No,
                    embedding_matrix,
                    ndim = 100,
                    tpu_enabled=0,
                    num_training_samples=1024,
                    num_validation_samples = 256,
                    num_epochs = 50,
                    batch_size = 10):
    num_training_samples = int((num_training_samples//128)*128)
    num_validation_samples = int((num_validation_samples//128)*128)
    #get the experiment details
    ExperimentNo = 'Experiment'+str(Experiment_No)
    num_unit_gru = Experiment_Dic[ExperimentNo]['num_unit_gru']
    num_layers_gru = Experiment_Dic[ExperimentNo]['num_layers_gru']
    num_episodes = Experiment_Dic[ExperimentNo]['num_episodes']
    num_dense_layer_feasibility_units = Experiment_Dic[ExperimentNo]['num_dense_layer_feasibility_units']
    dropout_rate = Experiment_Dic[ExperimentNo]['dropout_rate']
    num_dense_layers_feasibility = Experiment_Dic[ExperimentNo]['num_dense_layers_feasibility']
    num_episodic_network_unit = Experiment_Dic[ExperimentNo]['num_episodic_network_unit']
    learning_rate = Experiment_Dic[ExperimentNo]['learning_rate']
    l1_regularizer_weight = Experiment_Dic[ExperimentNo]['l1_regularizer_weight']
    l2_regularizer_weight = Experiment_Dic[ExperimentNo]['l2_regularizer_weight']
        
    if tpu_enabled==0:
        #When GPU ENABLED
        answer_model,\
        encoder_model,\
        feasibility_model = create_models(
                                      embedding_matrix = embedding_matrix,
                                      max_context_length = context_maxlen,
                                      max_question_length = question_maxlen,
                                      max_answer_length = answer_maxlen,
                                      num_unit_gru = num_unit_gru,
                                      num_layers_gru = num_layers_gru,
                                      ndim =ndim,
                                      num_episodes = num_episodes,
                                      num_dense_layer_feasibility_units = num_dense_layer_feasibility_units,
                                      dropout_rate = dropout_rate,
                                      num_dense_layers_feasibility = num_dense_layers_feasibility,
                                      num_episodic_network_unit = num_episodic_network_unit,
                                      l1_regularizer_weight = l1_regularizer_weight,
                                      l2_regularizer_weight = l2_regularizer_weight)

        adam_optim = keras.optimizers.Adam(learning_rate=learning_rate)
        #encoder_model.compile(optimizer=adam_optim,
        #                      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        #                      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
        #encoder_model.summary()
        #answer_model.compile(optimizer=adam_optim,
        #                           loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        #                           metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
        #                        )
        #
        #answer_model.summary()
        feasibility_model.compile(optimizer=adam_optim,
                                   loss=tf.keras.losses.BinaryCrossentropy(),
                                   metrics=[tf.keras.metrics.BinaryAccuracy()]
                                   )
        #feasibility_model.summary()
    else: 
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' +
                                                                     os.environ['COLAB_TPU_ADDR'])
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        batch_size = 128*8
        with strategy.scope():
            answer_model,\
            encoder_model,\
            feasibility_model = create_models(
                                          embedding_matrix = embedding_matrix,
                                          max_context_length = context_maxlen,
                                          max_question_length = question_maxlen,
                                          max_answer_length = answer_maxlen,
                                          num_unit_gru = num_unit_gru,
                                          num_layers_gru = num_layers_gru,
                                          ndim =ndim,
                                          num_episodes = num_episodes,
                                          num_dense_layer_feasibility_units = 
                                                        num_dense_layer_feasibility_units,
                                          dropout_rate = dropout_rate,
                                          num_dense_layers_feasibility = num_dense_layers_feasibility,
                                          num_episodic_network_unit = num_episodic_network_unit,
                                         l1_regularizer_weight = l1_regularizer_weight,
                                         l2_regularizer_weight = l2_regularizer_weight)

            adam_optim = keras.optimizers.Adam(learning_rate=learning_rate)
            #encoder_model.compile(optimizer=adam_optim,
            #                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            #                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
            #encoder_model.summary()
            #
            #answer_model.compile(optimizer=adam_optim,
            #                           loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            #                           metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
            #                    )

            #answer_model.summary()
            feasibility_model.compile(optimizer=adam_optim,
                                       loss=tf.keras.losses.BinaryCrossentropy(),
                                       metrics=[tf.keras.metrics.BinaryAccuracy()]
                                       )
            #feasibility_model.summary()
    #Train the Answer Model and Encoder Model
    answer_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    answer_loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
    val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
    @tf.function
    def answer_loss_function(real, pred):
        mask = tf.math.logical_not(tf.math.equal(real, 0))
        loss_ = answer_loss_object(real, pred)
        mask = tf.cast(mask, dtype=loss_.dtype)
        loss_ *= mask
        return tf.reduce_mean(loss_)

    @tf.function
    def answer_train_step(inp,ques,targ,encoder_model,answer_model):
        loss = 0
        with tf.GradientTape() as tape:
            episodic_memory,concatenated_tensor,question_output,context_output = encoder_model(inp,ques)
            answer_hidden = question_output
            dec_input = tf.expand_dims(targ[:,0], 1)
            for t in range(1, targ.shape[1]):
                predictions, answer_hidden, _ = answer_model(dec_input,
                                                             answer_hidden,
                                                             question_output,
                                                             context_output,
                                                             episodic_memory)
                loss += answer_loss_function(targ[:, t], predictions)
                train_acc_metric(targ[:, t],predictions)
                dec_input = tf.expand_dims(targ[:, t], 1)
                episodic_memory = None
            batch_loss = (loss / int(targ.shape[1]))
            variables = encoder_model.trainable_variables + answer_model.trainable_variables
            gradients = tape.gradient(loss, variables)
            answer_optimizer.apply_gradients(zip(gradients, variables))
            return batch_loss

    @tf.function
    def answer_val_step(inp,ques,targ,encoder_model,answer_model):
        loss = 0
        episodic_memory,concatenated_tensor,question_output,context_output = encoder_model(inp,ques)
        answer_hidden = question_output
        dec_input = tf.expand_dims(targ[:,0], 1)
        for t in range(1, targ.shape[1]):
            predictions, answer_hidden, _ = answer_model(dec_input,
                                                         answer_hidden,
                                                         question_output,
                                                         context_output,
                                                         episodic_memory)
            loss += answer_loss_function(targ[:, t], predictions)
            val_acc_metric(targ[:, t],predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
            episodic_memory = None
        batch_loss = (loss / int(targ.shape[1]))
        return batch_loss
    #Create batches for training
    
    TRAIN_BUFFER_SIZE = train_context_padded_seq[:num_training_samples].shape[0]
    VAL_BUFFER_SIZE = val_context_padded_seq[:num_validation_samples].shape[0]
    steps_per_epoch = TRAIN_BUFFER_SIZE//batch_size
    steps_per_epoch_val = VAL_BUFFER_SIZE//batch_size
    train_dataset = tf.data.Dataset.from_tensor_slices((train_context_padded_seq[:num_training_samples],
                                                        train_question_seq_padded[:num_training_samples],
                                                        train_answer_seq_padded[:num_training_samples],
                                                        train_answer_input_seq_padded[:num_training_samples],
                                                        train_answer_target_seq_padded[:num_training_samples],
                                                        train_answer_impossible[:num_training_samples]))\
                                   .shuffle(TRAIN_BUFFER_SIZE,reshuffle_each_iteration=True)
    train_dataset = train_dataset.batch(batch_size, drop_remainder=True)

    val_dataset = tf.data.Dataset.from_tensor_slices((val_context_padded_seq[:num_validation_samples],
                                                      val_question_seq_padded[:num_validation_samples],
                                                      val_answer_seq_padded[:num_validation_samples],
                                                      val_answer_input_seq_padded[:num_validation_samples],
                                                      val_answer_target_seq_padded[:num_validation_samples],
                                                      val_answer_impossible[:num_validation_samples]))
    val_dataset = val_dataset.batch(batch_size,drop_remainder=True)
    
    #Run Epochs
    
    history_answer_model = {'loss':[],
                            'sparse_categorical_accuracy':[],
                            'val_loss':[],
                            'val_sparse_categorical_accuracy':[]}

    print("Training the answer model:")
    for epoch in range(num_epochs):
        start = time.time()

        total_loss = 0
        total_val_loss = 0

        for (batch, (batch_train_context_padded_seq,
                     batch_train_question_seq_padded,
                     batch_train_answer_seq_padded,
                     batch_train_answer_input_seq_padded,
                     batch_train_answer_target_seq_padded,
                     batch_train_answer_impossible)) in enumerate(train_dataset.take(steps_per_epoch)):
            batch_loss = answer_train_step(batch_train_context_padded_seq,
                                           batch_train_question_seq_padded,
                                           batch_train_answer_input_seq_padded,
                                           encoder_model,
                                           answer_model)
            total_loss += batch_loss
            if batch % 50 == 0:
                print('=t',end='')

        epoch_training_loss = total_loss / steps_per_epoch
        epoch_training_accuracy = train_acc_metric.result()
        train_acc_metric.reset_states()
        print('')
        print('Epoch {} Train Loss {:.4f}'.format(epoch + 1,epoch_training_loss),end=' ')
        print('Epoch {} Train Accuracy {:.4f}'.format(epoch + 1,epoch_training_accuracy))
        
        
        for (batch, (batch_val_context_padded_seq,
                     batch_val_question_seq_padded,
                     batch_val_answer_seq_padded,
                     batch_val_answer_input_seq_padded,
                     batch_val_answer_target_seq_padded,
                     batch_val_answer_impossible)) in enumerate(val_dataset.take(steps_per_epoch_val)):
            batch_loss = answer_val_step(batch_val_context_padded_seq,
                                         batch_val_question_seq_padded,
                                         batch_val_answer_input_seq_padded,
                                         encoder_model,
                                         answer_model)
            total_val_loss += batch_loss
            if batch % 50 == 0:
                print('=v',end='')

        epoch_val_loss = total_val_loss / steps_per_epoch_val
        epoch_val_accuracy = val_acc_metric.result()
        val_acc_metric.reset_states()
        print('')
        print('Epoch {} Validation Loss {:.4f}'.format(epoch + 1,epoch_val_loss),end=' ')
        print('Epoch {} Validation Accuracy {:.4f}'.format(epoch + 1,epoch_val_accuracy))
        
        
        history_answer_model['loss'].append(epoch_training_loss)
        history_answer_model['sparse_categorical_accuracy'].append(epoch_training_accuracy)
        history_answer_model['val_loss'].append(epoch_val_loss)
        history_answer_model['val_sparse_categorical_accuracy'].append(epoch_val_accuracy)
        
        print('Time taken for epoch {} sec\n'.format(time.time() - start))

    
    answer_model.save_weights(ExperimentNo+'attention_model_answer_model.h5')
    encoder_model.save_weights(ExperimentNo+'attention_model_encoder_model.h5')
    with open(ExperimentNo+'attention_model_'+'history_answer_model', 'wb') as file_history:
        pickle.dump(history_answer_model, file_history)
        
    
    #Train the Feasibility Model
    _,encoder_prediction,encoder_prediction_question,\
    encoder_prediction_contexts = encoder_model(train_context_padded_seq[:num_training_samples],
                                             train_question_seq_padded[:num_training_samples])
    _,encoder_validation_prediction,encoder_validation_prediction_question,\
    encoder_validation_prediction_contexts = encoder_model(val_context_padded_seq[:num_validation_samples],
                                                         val_question_seq_padded[:num_validation_samples])
    print("training the feasibility model")
    history_feasibility_model = feasibility_model.fit([encoder_prediction,
                                                       encoder_prediction_contexts,
                                                       encoder_prediction_question],
                                                      train_answer_impossible[:num_training_samples],
                                                      epochs=num_epochs,
                                                      batch_size=batch_size,
                                                      validation_data = 
                                                            ([encoder_validation_prediction,
                                                              encoder_validation_prediction_contexts,
                                                              encoder_validation_prediction_question],
                                                             val_answer_impossible[:num_validation_samples])
                                                      )



    feasibility_model.save(ExperimentNo+'attention_model_feasibility_model.h5')
    with open(ExperimentNo+'attention_model_'+'history_feasibility_model', 'wb') as file_history:
        pickle.dump(history_feasibility_model.history, file_history)
    

In [13]:
def create_inference_model(Experiment_Dic,
                           Experiment_No,
                           embedding_matrix,
                           ndim = 100):
    ExperimentNo = 'Experiment'+str(Experiment_No)
    num_unit_gru = Experiment_Dic[ExperimentNo]['num_unit_gru']
    num_layers_gru = Experiment_Dic[ExperimentNo]['num_layers_gru']
    num_episodes = Experiment_Dic[ExperimentNo]['num_episodes']
    num_dense_layer_feasibility_units = Experiment_Dic[ExperimentNo]['num_dense_layer_feasibility_units']
    dropout_rate = Experiment_Dic[ExperimentNo]['dropout_rate']
    num_dense_layers_feasibility = Experiment_Dic[ExperimentNo]['num_dense_layers_feasibility']
    num_episodic_network_unit = Experiment_Dic[ExperimentNo]['num_episodic_network_unit']
    learning_rate = Experiment_Dic[ExperimentNo]['learning_rate']
    l1_regularizer_weight = Experiment_Dic[ExperimentNo]['l1_regularizer_weight']
    l2_regularizer_weight = Experiment_Dic[ExperimentNo]['l2_regularizer_weight']
    inference_answer_model,\
    inference_encoder_model,\
    inference_feasibility_model = create_models(
                                          embedding_matrix = embedding_matrix,
                                          max_context_length = context_maxlen,
                                          max_question_length = question_maxlen,
                                          max_answer_length = answer_maxlen,
                                          num_unit_gru = num_unit_gru,
                                          num_layers_gru = num_layers_gru,
                                          ndim =ndim,
                                          num_episodes = num_episodes,
                                          num_dense_layer_feasibility_units = num_dense_layer_feasibility_units,
                                          dropout_rate = dropout_rate,
                                          num_dense_layers_feasibility = num_dense_layers_feasibility,
                                          num_episodic_network_unit = num_episodic_network_unit,
                                          l1_regularizer_weight = l1_regularizer_weight,
                                          l2_regularizer_weight = l2_regularizer_weight )

    # train on 1 row so that weights can be loaded 
    #Train the Answer Model and Encoder Model
    inference_answer_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    inference_answer_loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    @tf.function
    def inference_answer_loss_function(real, pred):
        mask = tf.math.logical_not(tf.math.equal(real, 0))
        loss_ = inference_answer_loss_object(real, pred)
        mask = tf.cast(mask, dtype=loss_.dtype)
        loss_ *= mask
        return tf.reduce_mean(loss_)

    @tf.function
    def inference_answer_train_step(inp,ques,targ,encoder_model,answer_model):
        loss = 0
        with tf.GradientTape() as tape:
            episodic_memory,concatenated_tensor,question_output,context_output = encoder_model(inp,ques)
            answer_hidden = question_output
            dec_input = tf.expand_dims(targ[:,0], 1)
            for t in range(1, targ.shape[1]):
                predictions, answer_hidden, _ = answer_model(dec_input,
                                                             answer_hidden,
                                                             question_output,
                                                             context_output,
                                                             episodic_memory)
                loss += inference_answer_loss_function(targ[:, t], predictions)
                #train_acc_metric(targ[:, t],predictions)
                dec_input = tf.expand_dims(targ[:, t], 1)
                episodic_memory = None
            batch_loss = (loss / int(targ.shape[1]))
            variables = encoder_model.trainable_variables + answer_model.trainable_variables
            gradients = tape.gradient(loss, variables)
            inference_answer_optimizer.apply_gradients(zip(gradients, variables))
            return batch_loss

    inference_answer_train_step(train_context_padded_seq[:1],
                              train_question_seq_padded[:1],
                              train_answer_input_seq_padded[:1],
                              inference_encoder_model,
                              inference_answer_model)
        
    inference_answer_model.load_weights(ExperimentNo+'attention_model_answer_model.h5')
    inference_encoder_model.load_weights(ExperimentNo+'attention_model_encoder_model.h5')
    inference_feasibility_model.load_weights(ExperimentNo+'attention_model_feasibility_model.h5')
    return (inference_answer_model,inference_encoder_model,inference_feasibility_model)

In [14]:
#Run Experiments
#change the num_training_samples and num_validation_samples make sure its multiple of 128 when running on tpu
#change to tpu_enabled = 1 when running on tpu 
#Change the batch_size and epochs too
run_experiments(Experiment_Dic=Experiment_Dic,
                Experiment_No=0,
                embedding_matrix=embedding_matrix,
                ndim = 100,
                tpu_enabled=0,
                num_training_samples=2048,
                num_validation_samples = 256,
                num_epochs = 100,
                batch_size = 20)

Training the answer model:
=t=t=t
Epoch 1 Train Loss 2.1799 Epoch 1 Train Accuracy 0.0868
=v
Epoch 1 Validation Loss 1.7453 Epoch 1 Validation Accuracy 0.0864
Time taken for epoch 152.1944534778595 sec

=t=t=t
Epoch 2 Train Loss 1.5025 Epoch 2 Train Accuracy 0.0883
=v
Epoch 2 Validation Loss 1.7725 Epoch 2 Validation Accuracy 0.0864
Time taken for epoch 106.2917423248291 sec

=t=t=t
Epoch 3 Train Loss 1.4343 Epoch 3 Train Accuracy 0.0883
=v
Epoch 3 Validation Loss 1.8065 Epoch 3 Validation Accuracy 0.0864
Time taken for epoch 105.81519865989685 sec

=t=t=t
Epoch 4 Train Loss 1.4088 Epoch 4 Train Accuracy 0.0884
=v
Epoch 4 Validation Loss 1.8318 Epoch 4 Validation Accuracy 0.0864
Time taken for epoch 105.52970695495605 sec

=t=t=t
Epoch 5 Train Loss 1.4045 Epoch 5 Train Accuracy 0.0883
=v
Epoch 5 Validation Loss 1.8568 Epoch 5 Validation Accuracy 0.0864
Time taken for epoch 105.77680325508118 sec

=t=t=t
Epoch 6 Train Loss 1.3975 Epoch 6 Train Accuracy 0.0883
=v
Epoch 6 Validation Loss 

=t=t=t
Epoch 46 Train Loss 0.2159 Epoch 46 Train Accuracy 0.2248
=v
Epoch 46 Validation Loss 3.6441 Epoch 46 Validation Accuracy 0.0485
Time taken for epoch 103.24605536460876 sec

=t=t=t
Epoch 47 Train Loss 0.1910 Epoch 47 Train Accuracy 0.2324
=v
Epoch 47 Validation Loss 3.6930 Epoch 47 Validation Accuracy 0.0519
Time taken for epoch 103.09227156639099 sec

=t=t=t
Epoch 48 Train Loss 0.1679 Epoch 48 Train Accuracy 0.2382
=v
Epoch 48 Validation Loss 3.7383 Epoch 48 Validation Accuracy 0.0496
Time taken for epoch 103.0259485244751 sec

=t=t=t
Epoch 49 Train Loss 0.1506 Epoch 49 Train Accuracy 0.2427
=v
Epoch 49 Validation Loss 3.7814 Epoch 49 Validation Accuracy 0.0492
Time taken for epoch 102.75559735298157 sec

=t=t=t
Epoch 50 Train Loss 0.1338 Epoch 50 Train Accuracy 0.2481
=v
Epoch 50 Validation Loss 3.8533 Epoch 50 Validation Accuracy 0.0496
Time taken for epoch 102.92084646224976 sec

=t=t=t
Epoch 51 Train Loss 0.1197 Epoch 51 Train Accuracy 0.2505
=v
Epoch 51 Validation Loss 3.9

=v
Epoch 91 Validation Loss 4.8326 Epoch 91 Validation Accuracy 0.0477
Time taken for epoch 103.09772396087646 sec

=t=t=t
Epoch 92 Train Loss 0.0060 Epoch 92 Train Accuracy 0.2716
=v
Epoch 92 Validation Loss 4.8299 Epoch 92 Validation Accuracy 0.0473
Time taken for epoch 102.88365936279297 sec

=t=t=t
Epoch 93 Train Loss 0.0059 Epoch 93 Train Accuracy 0.2720
=v
Epoch 93 Validation Loss 4.8679 Epoch 93 Validation Accuracy 0.0492
Time taken for epoch 102.93502712249756 sec

=t=t=t
Epoch 94 Train Loss 0.0058 Epoch 94 Train Accuracy 0.2721
=v
Epoch 94 Validation Loss 4.8538 Epoch 94 Validation Accuracy 0.0481
Time taken for epoch 102.74119758605957 sec

=t=t=t
Epoch 95 Train Loss 0.0052 Epoch 95 Train Accuracy 0.2715
=v
Epoch 95 Validation Loss 4.9184 Epoch 95 Validation Accuracy 0.0485
Time taken for epoch 102.59345841407776 sec

=t=t=t
Epoch 96 Train Loss 0.0051 Epoch 96 Train Accuracy 0.2721
=v
Epoch 96 Validation Loss 4.8750 Epoch 96 Validation Accuracy 0.0477
Time taken for epoch 102

Epoch 38/100
2048/2048 [==============================] - 0s 203us/sample - loss: 0.2977 - binary_accuracy: 0.9580 - val_loss: 1.8524 - val_binary_accuracy: 0.5781
Epoch 39/100
2048/2048 [==============================] - 0s 204us/sample - loss: 0.3185 - binary_accuracy: 0.9565 - val_loss: 2.1392 - val_binary_accuracy: 0.5820
Epoch 40/100
2048/2048 [==============================] - 0s 203us/sample - loss: 0.2840 - binary_accuracy: 0.9609 - val_loss: 1.6128 - val_binary_accuracy: 0.5781
Epoch 41/100
2048/2048 [==============================] - 0s 203us/sample - loss: 0.3315 - binary_accuracy: 0.9478 - val_loss: 2.0332 - val_binary_accuracy: 0.5977
Epoch 42/100
2048/2048 [==============================] - 0s 203us/sample - loss: 0.3556 - binary_accuracy: 0.9448 - val_loss: 2.0025 - val_binary_accuracy: 0.5820
Epoch 43/100
2048/2048 [==============================] - 0s 202us/sample - loss: 0.3308 - binary_accuracy: 0.9473 - val_loss: 1.7422 - val_binary_accuracy: 0.5703
Epoch 44/100
204

Epoch 88/100
2048/2048 [==============================] - 0s 203us/sample - loss: 0.3236 - binary_accuracy: 0.9482 - val_loss: 1.7511 - val_binary_accuracy: 0.5508
Epoch 89/100
2048/2048 [==============================] - 0s 204us/sample - loss: 0.3293 - binary_accuracy: 0.9521 - val_loss: 1.9355 - val_binary_accuracy: 0.5977
Epoch 90/100
2048/2048 [==============================] - 0s 200us/sample - loss: 0.3412 - binary_accuracy: 0.9468 - val_loss: 2.3229 - val_binary_accuracy: 0.5820
Epoch 91/100
2048/2048 [==============================] - 0s 201us/sample - loss: 0.3169 - binary_accuracy: 0.9478 - val_loss: 1.9576 - val_binary_accuracy: 0.5898
Epoch 92/100
2048/2048 [==============================] - 0s 202us/sample - loss: 0.3259 - binary_accuracy: 0.9443 - val_loss: 2.1103 - val_binary_accuracy: 0.5859
Epoch 93/100
2048/2048 [==============================] - 0s 202us/sample - loss: 0.3195 - binary_accuracy: 0.9536 - val_loss: 1.7434 - val_binary_accuracy: 0.5977
Epoch 94/100
204

In [15]:
#Run Inference
inference_answer_model,\
inference_encoder_model,\
inference_feasibility_model = create_inference_model(Experiment_Dic=Experiment_Dic,
                                                     Experiment_No=0,
                                                     embedding_matrix=embedding_matrix,
                                                     ndim = 100)

In [22]:
for seq_index in range(8000,8010):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    context_input_seq = train_context_padded_seq[seq_index: seq_index+ 1]
    question_input_seq = train_question_seq_padded[seq_index: seq_index + 1]
    decoded_sentence = decode_sentence(context_input_seq,
                                       question_input_seq,
                                       inference_encoder_model,
                                       inference_answer_model)
    print("question:",' '.join([id_vocab.get(i) for i in train_question_seq_padded[seq_index].tolist() if i !=0]))
    print('Predicted Answer:', decoded_sentence)
    act_answer = ' '.join([id_vocab.get(i) for i in train_answer_input_seq_padded[seq_index].tolist() if i !=0])
    print('Actual answer:',act_answer)

question: how many members are on the city commission
Predicted Answer: 1 000 </s> 
Actual answer: <s> five </s>
question: how does ps4 limit data use
Predicted Answer: </s> 
Actual answer: <s> </s>
question: what game was released in north america but not europe
Predicted Answer: </s> 
Actual answer: <s> </s>
question: how much was public expenditure on the island in 2001 2002
Predicted Answer: 1 907 bc </s> 
Actual answer: <s> £10 million </s>
question: what generates output streams from each endpoint
Predicted Answer: free </s> 
Actual answer: <s> the mp </s>
question: when was the london jewish forum established
Predicted Answer: may new england </s> 
Actual answer: <s> 2006 </s>
question: what kingdom came to power in assam
Predicted Answer: kokhir co uk </s> 
Actual answer: <s> ahom kingdom </s>
question: what mathematician advance greek science
Predicted Answer: antibodies the greens </s> 
Actual answer: <s> greek science </s>
question: how long did the lenin square protest last